Name: Feyzullah Alim Kalyoncu

2015751042

I hereby declare that I observed the honour code of the university when preparing the homework.

## Programming Homework 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

('a', '0.0002835')
('b', '0.0228302')
('c', '0.0369041')
('d', '0.0426290')
('e', '0.0012216')
('f', '0.0075739')
('g', '0.0171385')
('h', '0.0014659')
('i', '0.0372661')
('j', '0.0002353')
('k', '0.0110124')
('l', '0.0778259')
('m', '0.0260757')
('n', '0.2145354')
('o', '0.0005459')
('p', '0.0195213')
('q', '0.0001749')
('r', '0.1104770')
('s', '0.0934290')
('t', '0.1317960')
('u', '0.0098029')
('v', '0.0306574')
('w', '0.0088799')
('x', '0.0009562')
('y', '0.0233701')
('z', '0.0018701')
('.', '0.0715219')


In [3]:
import numpy as np

In [4]:
#inverse mapping of characters is generated to use 
inverse_map = dict((y,x) for x,y in letter2idx.iteritems())

In [5]:
#string list to numpy array conversion of transition matrix
t = []
letters = []
for i in inverse_map:
    letters.append(inverse_map[i])
    t.append([float(i) for i in T[letter2idx[inverse_map[i]]]])
transition = np.array(t)

In [6]:
#Q1
#For given N text is generated with using transition matrix
N = 10
#Starting letter is '.'
prev_letter = '.'
generated_str = []
for i in range(0,N):
    prev_letter = np.random.choice(letters,1, p = transition[letter2idx[prev_letter]] / np.sum(transition[letter2idx[prev_letter]]) )[0]
    generated_str.append(prev_letter)
print ''.join(generated_str)

wread.ss.i


In [7]:
display(Latex(r"$x_{0} -> x_{1} -> x_{2} -> ........ -> x_{N} -> x_{N+1}$") ) 
display(Latex(r"$We\hspace{0.05cm}know\hspace{0.1cm}x_{0}\hspace{0.05cm}and\hspace{0.1cm}x_{N+1}\hspace{0.1cm}and\hspace{0.1cm}we\hspace{0.1cm}are\hspace{0.1cm}trying\hspace{0.1cm}to\hspace{0.1cm}estimate\hspace{0.1cm}the\hspace{0.1cm}characters\hspace{0.1cm}between\hspace{0.1cm}them$"))
display(Latex(r"$p(x_{1:N}  | x_0 = \widehat{x}_0, x_{N+1} = \widehat{x}_{N+1}) \propto$") ) 
display(Latex(r"$p(x_{N+1} = \widehat{x}_{N+1} | x_N,  x_0 = \widehat{x}_0)\hspace{0.1cm}\times\hspace{0.1cm}p(x_{N} | x_{N-1},  x_0 = \widehat{x}_0)\hspace{0.1cm}\times\hspace{0.1cm}\dots\hspace{0.1cm}\times\hspace{0.1cm}p(x_{2} | x_{1},  x_0 = \widehat{x}_0)\hspace{0.1cm}\times\hspace{0.1cm}p(x_{1} | x_0 = \widehat{x}_0) =$") )
display(Latex(r"$p(x_{N+1} = \widehat{x}_{N+1} | x_N) \hspace{0.1cm}\times\hspace{0.1cm} p(x_{N} | x_{N-1}) \hspace{0.1cm}\times\hspace{0.1cm}\dots\hspace{0.1cm}\times\hspace{0.1cm}p(x_{2} | x_{1}) \hspace{0.1cm}\times\hspace{0.1cm} p(x_{1} | x_0 = \widehat{x}_0)$") ) 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
test_strings

['th__br__n.f_x.',
 '_u_st__n_.to_be._nsw_r__',
 'i__at_._a_h_n_._e_r_i_g',
 'q___t.___z._____t.__.___.__.']

In [9]:
from numpy import linalg as LA

In [10]:
#Q2

In [11]:
for string in test_strings:
    prev_index = -1
    prev_letter = '.'
    str_list = list(string)
    # '.' is added to the end of the given sequence
    str_list.append('.')
    for idx, letter in enumerate(str_list):
        if letter == '_':
            #previous character and its index is found here
            for i in range(idx-1,-1,-1):
                if str_list[i] != '_':
                    prev_index = i
                    prev_letter = str_list[i]
                    break
            #next character and its index is found here
            for i in range(idx,len(str_list),+1):
                if str_list[i] != '_':
                    next_index = i
                    next_letter = str_list[i]
                    break
            # power is the power of the transition matrix that we need to find
            # as in the derivation given above, the next letter-row of powered matrix represents the part from x_N+1 to x_1
            power = next_index - prev_index -1
            powered = LA.matrix_power(transition, power)
            #multiplying prev letter column of transition matrix with calculated row of powered matrix gives us our final probabilities
            choices = np.multiply(transition[letter2idx[prev_letter]], powered[:,letter2idx[next_letter]])
            choices = choices / np.sum(choices)
            #character for given index is randomly chooses
            str_list[idx] = np.random.choice(letters,1, p = choices)[0]
        
    print ''.join(str_list[0:len(str_list)-1])

theabrcon.fex.
ourstlond.toube.answar.t
ig.ate..athenm.te.raing
qur.t..cez.th.t.t.wh.y.e.rf.


In [12]:
#Q3

In [13]:
import math

In [14]:
prob = 0
for string in test_strings:
    prev_index = -1
    prev_letter = '.'
    str_list = list(string)
    str_list.append('.')
    for idx, letter in enumerate(str_list):
        if letter == '_':
            
            for i in range(idx-1,-1,-1):
                if str_list[i] != '_':
                    prev_index = i
                    prev_letter = str_list[i]
                    break
            
            for i in range(idx,len(str_list),+1):
                if str_list[i] != '_':
                    next_index = i
                    next_letter = str_list[i]
                    break
                    
            power = next_index - prev_index -1
            powered = LA.matrix_power(transition, power)
            choices = np.multiply(transition[letter2idx[prev_letter]], powered[:,letter2idx[next_letter]])
            choices = choices / np.sum(choices)
            #everthing until this point is the same as above
            #here we take the argmax of probabilities array
            #this gives us the most possible sequence
            str_list[idx] = letters[np.argmax(choices)]
            #log of character probabilites added to find the probability of the maximum sequence
            prob += math.log10(choices[np.argmax(choices)])
    #log10 probability of most probable string
    print prob 
    print ''.join(str_list[0:len(str_list)-1])

-1.33516666773
the.br.an.fex.
-6.14251472448
oursthend.to.be.answered
-11.1960044007
in.ath.wathend.he.r.ing
-21.15161581
qur.t.thiz.the.at.an.the.an.


#Q4
We may get better results and more readable strings if we consider all of the given characters while estimating a gap.
However, this may also distort the results. The best way to find the most possible and logical strings is that
we should use words instead of characters (transition matrix is given for characters) to find most probable string. If we have a dictionary with given probabilities, we can estimate the words between two dots. Probabilities in this dictionary is like the transition matrix we have but it has words instead of characters. In this way, we can create a model that always finds a logical string which fits into the given charters and gaps